In [1]:
import pandas as pd
import re

# Read the list of filenames from the configuration file
with open('../file_list.txt', 'r', encoding='utf-8') as config_file:
    file_names = config_file.read().splitlines()

# Regex pattern to match the data format
pattern = r'\[(.*?)\] (.*?): (.*)'

# Initialize an empty list to store parsed data
datalist = []
stream_count = 0
# Iterate over each specified file
for file in file_names:
    full_path = f"../data/{file}"
    with open(full_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            match = re.match(pattern, line)
            if match:
                date, user, message = match.groups()
                datalist.append([date, user, message,stream_count])
    stream_count = stream_count + 1

# Create a DataFrame from the parsed data
data = pd.DataFrame(datalist, columns=["date", "user", "message","stream"])


In [2]:
data = data[data["user"] != "StreamElements"]
data = data[data["user"] != "Fossabot"]

In [3]:

data['date'] = pd.to_datetime(data['date'])

data = data[data['date'].dt.year == 2025]



In [4]:
data["user"] = data["user"].replace("Banties1g", "banties_x")
data["user"] = data["user"].replace("banties1g", "banties_x")
data["user"] = data["user"].replace("chili_poe", "chili_con_bacon")
data["user"] = data["user"].replace("CHILI_POE", "chili_con_bacon")
data["user"] = data["user"].replace("chili_conbacon", "chili_con_bacon")
data["user"] = data["user"].replace("Wirelesss_", "W1r3lesss")
data["user"] = data["user"].replace("treklul", "trek44_")
data["user"] = data["user"].replace("ttrek_", "trek44_")
data["user"] = data["user"].replace("trek_x", "trek44_")
data["user"] = data["user"].replace("TriplesingleJ", "TripleSingleJames")
data["user"] = data["user"].replace("uuccugr", "uwu_cougar")
data["user"] = data["user"].replace("uuccugr", "uuccugr_")
data["user"] = data["user"].replace("StanIV4_", "stan_iv4")
data["user"] = data["user"].replace("Muuskie2", "Muuskie")
data["user"] = data["user"].replace("nishad_more1311", "nishad13")
data["user"] = data["user"].replace("softarballt", "softarr")
data["user"] = data["user"].replace("softarballtt23", "softarr")
data["user"] = data["user"].replace("bonkwiththefunk", "bonk67")

In [5]:
from collections import defaultdict

# Get all unique usernames
unique_users = data['user'].unique()

# Create a mapping from lowercase username to all variants

user_variants = defaultdict(set)
for user in unique_users:
    user_variants[user.lower()].add(user)

# Find usernames with different capitalization
duplicate_users = {k: v for k, v in user_variants.items() if len(v) > 1}

In [6]:
# Create a mapping from all variants to the canonical (sorted first) variant
variant_map = {}
for variants in duplicate_users.values():
    sorted_variants = sorted(variants)
    canonical = sorted_variants[0]
    for v in variants:
        variant_map[v] = canonical

# Replace usernames in 'user' column
data['user'] = data['user'].apply(lambda u: variant_map.get(u, u))

In [7]:
# Count the number of messages per user
message_counts = data.groupby("user")["message"].count()

# Filter users with 25 or more messages
users_with_25_or_more = message_counts[message_counts >= 25].index

# Filter the original DataFrame to keep only these users
data = data[data["user"].isin(users_with_25_or_more)]

In [8]:
from collections import defaultdict

# Get all unique usernames
unique_users = data['user'].unique()

# Create a mapping from lowercase username to all variants

user_variants = defaultdict(set)
for user in unique_users:
    user_variants[user.lower()].add(user)

# Find usernames with different capitalization
duplicate_users = {k: v for k, v in user_variants.items() if len(v) > 1}

In [9]:
# Create a mapping from all variants to the canonical (sorted first) variant
variant_map = {}
for variants in duplicate_users.values():
    sorted_variants = sorted(variants)
    canonical = sorted_variants[0]
    for v in variants:
        variant_map[v] = canonical

# Replace usernames in 'user' column
data['user'] = data['user'].apply(lambda u: variant_map.get(u, u))

In [10]:
# Convert date to datetime format
data["date"] = pd.to_datetime(data["date"])



In [11]:
# Truncate datetime to just the day (removing time)
data["day"] = data["date"].dt.date  # Extract only the date part



In [12]:
# Group by 'day' and 'user' and calculate the message count per day per user
data["message_count"] = 1  # Assign 1 for each message to count them
daily_counts = data.groupby(["day", "user"])["message_count"].count().reset_index()



In [13]:
# Pivot the table to create a user-wise table for each day
pivot_table = daily_counts.pivot(index="day", columns="user", values="message_count").fillna(0)


In [14]:

# Add a cumulative sum for each user across the days
cumulative_pivot = pivot_table.cumsum()


In [15]:

# Print the result
print(cumulative_pivot)

user        00cad  00skysea00  00yopop  010justwatch  01gomesss  06RoB  \
day                                                                      
2025-01-01    0.0         0.0      0.0           0.0        0.0    0.0   
2025-01-03    0.0         0.0      0.0           0.0        0.0    0.0   
2025-01-04    0.0         0.0      0.0           0.0        0.0    0.0   
2025-01-05    0.0         0.0      0.0           0.0        0.0    0.0   
2025-01-06    0.0         0.0      0.0           0.0        0.0    0.0   
...           ...         ...      ...           ...        ...    ...   
2025-12-26   55.0        27.0     55.0          55.0       47.0  325.0   
2025-12-27   55.0        27.0     55.0          55.0       47.0  325.0   
2025-12-28   55.0        27.0     55.0          55.0       47.0  325.0   
2025-12-30   55.0        27.0     55.0          55.0       47.0  325.0   
2025-12-31   55.0        27.0     55.0          55.0       47.0  328.0   

user        07moki  0SHADOWIX  0_0and

In [16]:
pivot_data_cleaned_transposed = cumulative_pivot.T
pivot_data_cleaned_transposed = cumulative_pivot.T.reset_index()

In [17]:
pivot_data_cleaned_transposed.head(5)
pivot_data_cleaned_transposed.tail(5)

day,user,2025-01-01,2025-01-03,2025-01-04,2025-01-05,2025-01-06,2025-01-08,2025-01-09,2025-01-10,2025-01-11,...,2025-12-20,2025-12-21,2025-12-22,2025-12-23,2025-12-25,2025-12-26,2025-12-27,2025-12-28,2025-12-30,2025-12-31
6994,기분조앙,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0
6995,너의이탈리아아저씨,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0
6996,빵댕이ㅋ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,22.0,22.0,22.0,22.0,26.0,28.0,28.0,29.0,30.0,30.0
6997,안톤958,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0
6998,알래스카해달,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4790.0,4882.0,4882.0,4890.0,4936.0,4998.0,5041.0,5091.0,5136.0,5322.0


In [18]:
# --- Step 1 & 2: Calculate rank and identify top 20 users ---

# Calculate rank for each date (column) in descending order.
# The user with the highest cumulative sum gets rank 1.
ranked_df = pivot_data_cleaned_transposed.rank(axis=0, ascending=False, method='min')

# Identify users who were in the top 20 on at least one date.
# This creates a boolean Series where True means the user was in the top 20 at least once.
users_in_top_15_at_least_once = (ranked_df <= 20).any(axis=1)

# Get the list of users (their index labels) who meet the criteria.
users_to_keep = users_in_top_15_at_least_once[users_in_top_15_at_least_once].index

# --- Step 3: Filter the DataFrame ---

# Create a new DataFrame containing only the users who were in the top 20 at least once.
filtered_users_df = pivot_data_cleaned_transposed.loc[users_to_keep]

print("Original DataFrame shape:", pivot_data_cleaned_transposed.shape)
print("Filtered DataFrame shape:", filtered_users_df.shape)

Original DataFrame shape: (6999, 309)
Filtered DataFrame shape: (70, 309)


In [19]:
filtered_users_df.head()

day,user,2025-01-01,2025-01-03,2025-01-04,2025-01-05,2025-01-06,2025-01-08,2025-01-09,2025-01-10,2025-01-11,...,2025-12-20,2025-12-21,2025-12-22,2025-12-23,2025-12-25,2025-12-26,2025-12-27,2025-12-28,2025-12-30,2025-12-31
28,1206paul_,51.0,109.0,172.0,238.0,309.0,415.0,468.0,516.0,610.0,...,7191.0,7219.0,7219.0,7219.0,7219.0,7229.0,7229.0,7259.0,7259.0,7261.0
218,Aloddin,0.0,114.0,124.0,146.0,173.0,191.0,230.0,237.0,248.0,...,4364.0,4364.0,4364.0,4367.0,4370.0,4370.0,4370.0,4370.0,4372.0,4373.0
221,Aluminiumminimumimmunity,39.0,218.0,251.0,334.0,368.0,389.0,413.0,426.0,474.0,...,13468.0,13525.0,13556.0,13631.0,13696.0,13767.0,13825.0,13919.0,13967.0,14213.0
321,BenXBari,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,49480.0,49480.0,49480.0,49573.0,49605.0,49605.0,49605.0,49624.0,49624.0,49754.0
460,CrazeE420xd,299.0,467.0,504.0,817.0,995.0,1337.0,1550.0,1795.0,1827.0,...,23982.0,24031.0,24031.0,24031.0,24031.0,24031.0,24031.0,24122.0,24122.0,24122.0


In [20]:
filtered_users_df.info

<bound method DataFrame.info of day                       user  2025-01-01  2025-01-03  2025-01-04  \
28                   1206paul_        51.0       109.0       172.0   
218                    Aloddin         0.0       114.0       124.0   
221   Aluminiumminimumimmunity        39.0       218.0       251.0   
321                   BenXBari         0.0         0.0         0.0   
460                CrazeE420xd       299.0       467.0       504.0   
...                        ...         ...         ...         ...   
6994                      기분조앙         0.0         0.0         0.0   
6995                 너의이탈리아아저씨         0.0         0.0         0.0   
6996                      빵댕이ㅋ         0.0         0.0         0.0   
6997                     안톤958         0.0         0.0         0.0   
6998                    알래스카해달         0.0         0.0         0.0   

day   2025-01-05  2025-01-06  2025-01-08  2025-01-09  2025-01-10  2025-01-11  \
28         238.0       309.0       415.0       

In [21]:
filtered_users_df.head()

day,user,2025-01-01,2025-01-03,2025-01-04,2025-01-05,2025-01-06,2025-01-08,2025-01-09,2025-01-10,2025-01-11,...,2025-12-20,2025-12-21,2025-12-22,2025-12-23,2025-12-25,2025-12-26,2025-12-27,2025-12-28,2025-12-30,2025-12-31
28,1206paul_,51.0,109.0,172.0,238.0,309.0,415.0,468.0,516.0,610.0,...,7191.0,7219.0,7219.0,7219.0,7219.0,7229.0,7229.0,7259.0,7259.0,7261.0
218,Aloddin,0.0,114.0,124.0,146.0,173.0,191.0,230.0,237.0,248.0,...,4364.0,4364.0,4364.0,4367.0,4370.0,4370.0,4370.0,4370.0,4372.0,4373.0
221,Aluminiumminimumimmunity,39.0,218.0,251.0,334.0,368.0,389.0,413.0,426.0,474.0,...,13468.0,13525.0,13556.0,13631.0,13696.0,13767.0,13825.0,13919.0,13967.0,14213.0
321,BenXBari,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,49480.0,49480.0,49480.0,49573.0,49605.0,49605.0,49605.0,49624.0,49624.0,49754.0
460,CrazeE420xd,299.0,467.0,504.0,817.0,995.0,1337.0,1550.0,1795.0,1827.0,...,23982.0,24031.0,24031.0,24031.0,24031.0,24031.0,24031.0,24122.0,24122.0,24122.0


In [22]:
filtered_users_df.to_excel('chattersRace2025.xlsx', sheet_name='Pivot Table')

In [23]:
data.shape

(2050563, 6)